本脚本进行under different scenario不同资产配置方法的蒙特卡洛随机模拟实验
Approach1:看似每周调一次仓位，实则每个交易日调一次，调至最近一次周度计算出的权重值
    Approach2:按照backtest中纯正周度实盘调仓

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mpl
import scipy.spatial.distance as ssd
import scipy.cluster.hierarchy as sch
import random

In [2]:
def tail_ratio(returns):
    """Determines the ratiro btw the right95% and left5% tail
     Input:
     returns: pd.Series
          daily/weekly/monthly returns of strategy, noncumulative
     Output:
     tail ratio describing how good or bad the relative tails will lead to
     """
    return np.abs(np.percentile(returns,95)) / np.abs(np.percentile(returns,5))

In [3]:
def computeVaR( ret_series, T = 10):
    #计算VaR: alpha = 0.01
    VaR_Tday = ret_series.mean() * T + (-2.33) * ret_series.std() * T**0.5
    
    return VaR_Tday

In [4]:
#Stage 1
def getClusterVar( cov,cItems ):
    #compute variance per cluster
    cov_ = cov.loc[ cItems,cItems ]  # matrix slice
    w_ = getIVP(cov_).reshape(-1,1)  #设定reshape后列数为1，行数-1代表自动匹配计算，w_为 X by 1 的风险分配权重
    cVar = np.dot( np.dot(w_.T, cov_),w_ )[0,0]   # 1byX * XbyX * Xby1
    return cVar

In [5]:
#Stage 2
def getQuasiDiag( link ):
    #执行Quasi-Diagonization的步骤，将因子暴露意义下相似的股票放到一起
    #sort clustered items by distance
    link = link.astype(int) #returns a copy of the array converted to the specified type
    #sortIx记录每一层的组分
    sortIx = pd.Series( [link[-1,0] , link[-1,1]] )
    numItems = link[-1,3]    # number of original items,最后一行为root
    while sortIx.max() >= numItems:
        sortIx.index = range(0 , sortIx.shape[0]*2 , 2)  #make space, step_length = 2
        df0 = sortIx[ sortIx>=numItems ]                 #find clusters，把代表cluster而非originalelement的取出来
        i = df0.index; j = df0.values - numItems
        sortIx[i] = link[j,0]     #item 1
        df0 = pd.Series( link[j,1] , index=i+1)  #循环变量在这里结束时+1
        sortIx = sortIx.append(df0)  #item 2
        sortIx = sortIx.sort_index()  #re-sort
        sortIx.index = range( sortIx.shape[0] ) #re-index
    return sortIx.tolist()

In [6]:
#Stage 3
def getRecBipart( cov, sortIx ):
    # Bottom-up: Define the variance of a continuous subset as the variance of an inverse-variance allocation
    # Top-down: Split allocations btw adjacent subsets in inverse proportion to their aggregated variances
    w = pd.Series(1 , index=sortIx)
    cItems = [sortIx]     #initialize all items in one cluster
    while len(cItems)>0:
        cItems = [ i[j:k] for i in cItems for j,k in ( (0,len(i)//2),(len(i)//2,len(i)) ) if len(i)>1 ]
        # above: bi-section
        for i in range(0, len(cItems), 2):    #parse in pairs
            cItems0 = cItems[i]   #cluster 1
            cItems1 = cItems[i+1] #cluster 2
            cVar0 = getClusterVar( cov,cItems0)
            cVar1 = getClusterVar( cov,cItems1)
            alpha = 1 - cVar0/(cVar0+cVar1)
            w[cItems0] *= alpha    #weight1
            w[cItems1] *= 1-alpha  #weight2
        
    return w   #返回最终标的权重Series

In [7]:
#Stage 0
def correlDist(corr):
    # Output a distance matrix based on correlation, where 0<= d[i,j] <=1
    #合理的距离测度，针对于由相关性定义的原始距离矩阵
    dist = ( (1-corr)/2. )**.5
    return dist

def CalEucliDist(vec1,vec2):  
    #Output euclidean distance
    #vec1,vec2分别为两个np.array
    eucli = np.sqrt( np.sum( np.square(vec1 - vec2) ) )  
    return eucli 

def correlDist_ofDist1(dist):
    # A distance matrix based on original dist matrix
    # Compute the Euclidean distance btw any two col-vectors of dist
    # Therefore, new mat is a distance defined over the ENTIRE metric space,\
    # Rather than a PARTICULAR cross-correlation pair!
    dist_ofDist = np.empty( shape(dist) )
    
    for i in range( 0,shape(dist)(1) ):
        for j in range( i+1,shape(dist)(1) ):
            dist_ofDist[i,j] = CalEucliDist( dist[:,i],dist[:,j] )
            dist_ofDist[j,i] = dist_ofDist[i,j]
            
    dist_ofDist = ssd.squareform(dist_ofDist)
    return dist_ofDist  # 转换为CondensedForm的

#Otherwise, 可以调用scipy包中高维空间距离矩阵的算法
def correlDist_ofDist2(dist):
    dist_ofDist = ssd.pdist( dist, 'euclidean' )  #返回CondensedForm，即只有上三角部分存成数组形式
    
    return dist_ofDist  # CondensedForm的

In [8]:
def plotCorrMatrix( path, corr, labels=None):
    # Heatmapping of the corr matri
    if labels is None: labels = []
    mpl.pcolor(corr)
    mpl.colorbar()
    mpl.yticks( np.arange(.5 , corr.shape[0]+.5) , labels )
    mpl.xticks( np.arange(.5 , corr.shape[0]+.5) , labels )
    mpl.savefig(path)
    mpl.clf(); mpl.close()   # reset pylab
    return

In [9]:
"""
def generateData( nObs, sLength,size0 ,size1 , mu0, sigma0, sigma1F):
    # Time series of correlated variables
    # Uncorrelated Data
    #sigma0为原始序列的波动率；sigma1F为在原始序列基础上，衍生序列的波动率额外增益
    x = np.random.normal( mu0 ,sigma0 , size=(nObs ,size0)) #each row is a variable
    
    # Creating correlation btw the variables
    #五条原始的独立序列，五条衍生出的相关序列
    cols = [ random.randint(0 , size0-1) for i in range(size1)]
    y = x[: , cols] + np.random.normal( 0 ,sigma0*sigma1F ,size=(nObs , len(cols)))
    #再把衍生序列叠加到原始序列边上
    x = np.append(x ,y ,axis =1)
    
    # add common random shock
    point = np.random.randint(sLength,nObs-1,size=2)
    x[point,cols[-1]] = np.array([-0.5 ,2])

    return x,cols
"""

'\ndef generateData( nObs, sLength,size0 ,size1 , mu0, sigma0, sigma1F):\n    # Time series of correlated variables\n    # Uncorrelated Data\n    #sigma0为原始序列的波动率；sigma1F为在原始序列基础上，衍生序列的波动率额外增益\n    x = np.random.normal( mu0 ,sigma0 , size=(nObs ,size0)) #each row is a variable\n    \n    # Creating correlation btw the variables\n    #五条原始的独立序列，五条衍生出的相关序列\n    cols = [ random.randint(0 , size0-1) for i in range(size1)]\n    y = x[: , cols] + np.random.normal( 0 ,sigma0*sigma1F ,size=(nObs , len(cols)))\n    #再把衍生序列叠加到原始序列边上\n    x = np.append(x ,y ,axis =1)\n    \n    # add common random shock\n    point = np.random.randint(sLength,nObs-1,size=2)\n    x[point,cols[-1]] = np.array([-0.5 ,2])\n\n    return x,cols\n'

In [10]:
def generateData( nObs, sLength,size0 ,size1 , mu0, sigma0, sigma1F):
    # Time series of correlated variables
    # Uncorrelated Data
    #sigma0为原始序列的波动率；sigma1F为在原始序列基础上，衍生序列的波动率额外增益
    x = np.random.normal( mu0 ,sigma0 , size=(nObs ,size0)) #each row is a variable
    
    # Creating correlation btw the variables
    #五条原始的独立序列，五条衍生出的相关序列
    cols = [ random.randint(0 , size0-1) for i in range(size1)]
    y = x[: , cols] + np.random.normal( 0 ,sigma0*sigma1F ,size=(nObs , len(cols)))
    #再把衍生序列叠加到原始序列边上
    x = np.append(x ,y ,axis =1)
    
    # add common random shock
    point = np.random.randint(sLength,nObs-1,size=2)
    x[np.ix_(point , [cols[0],size0])] = np.array([ [-0.5,-0.5],[2,2]])
    # add specific random shock
    point = np.random.randint(sLength,nObs-1, size=2)
    x[point,cols[-1]] = np.array([-0.5 ,2])

    return x,cols

In [11]:
def getHRP(cov, corr):
   
    #Construct a hierarchical portfolio
    corr,cov = pd.DataFrame(corr),pd.DataFrame(cov)
    dist = correlDist(corr)
    
    new_dist = correlDist_ofDist2(dist)
    
    link = sch.linkage(new_dist,'single')
    sortIx = getQuasiDiag(link)
    sortIx = corr.index[sortIx].tolist()
    hrp = getRecBipart(cov, sortIx)
    
    return hrp.sort_index()


def getIVP( cov , **kargs):
   
    # compute the inverse-variance portfolio
    ivp = 1 / np.diag(cov)   #ivp为一维数组
    ivp /= ivp.sum()
    return ivp               #最终方差大的部分ivp较小

描述了不同市场波动率程度和shock来源的特异性或普遍性
多次实验参数设置为：sigma0取0.02，0.08，0.20；  越大波动环境越强
              sigma1取0.05，0.20，0.50；  越大特异性shock越强
    mu0 = 0,因其表示收益序列的均值
    模拟次数iters = 600

In [12]:
def Simulation(numIters = 600 , nObs = 520, size0 = 5,size1 = 5, mu0 = 0, sigma0 = 0.02,\
         sigma1F = 0.05, sLength =260, rebal = 22): #总长520
    #monte carlo experiment 
    #默认参数为：最低波动率环境，特异性冲击最少
    stats = {'HRP': pd.Series(),'IVP': pd.Series(),'EQUAL':pd.Series(),\
             'HRP_VaR': pd.Series(),'IVP_VaR': pd.Series(),'EQUAL_VaR':pd.Series(),\
             'HRP_TailRatio':pd.Series(),'IVP_TailRatio':pd.Series(),'EQUAL_TailRatio':pd.Series()}
         #存储不同配置方法的实验统计结果
         #字典（key为方法，value为series）后来转为dataframe
    
    pointers = range(sLength ,nObs ,rebal) #从sLength起始，到nObs截止，步长为22天，月度
    for numIter in range(int(numIters)):
        print(numIter)
        
        #一、准备数据 for one single experiment
        x, cols = generateData(nObs ,sLength ,size0,size1, mu0 ,sigma0 , sigma1F)
        
        r = pd.DataFrame( columns = ['HRP','IVP','EQUAL'], index=range(sLength,nObs))

        
        #二、样本内资产组合
        for pointer in pointers:
            x_ = x[pointer - sLength : pointer]  #每段窗长为sLength，为pointer那天向前的一段时间
            cov_  = np.cov(x_ ,rowvar = 0)       #为时间序列协方差矩阵
            corr_ = np.corrcoef(x_, rowvar = 0)   #为时间序列相关性矩阵
          
            #三、样本外资产组合
            x_ = x[pointer : pointer+rebal]      #替换为样本外一个月
            w1 = getHRP(cov=cov_, corr=corr_)  #输入的为样本内参数，得到分配权重
            w2 = getIVP(cov=cov_, corr=corr_)
            size2 = size0+size1
            w2 = pd.Series(w2)
            w3 = pd.Series([1/size2,1/size2,1/size2,1/size2,1/size2,1/size2,1/size2,1/size2,1/size2,1/size2])
        
            r.loc[pointer:pointer + rebal - 1,'HRP'] = np.dot(x_, w1)
            r.loc[pointer:pointer + rebal - 1,'IVP'] = np.dot(x_, w2)
            r.loc[pointer:pointer + rebal - 1,'EQUAL'] = np.dot(x_, w3)
            
        # 四、 Evaluate and store results
            
        #不同方案 最终受益 和 99%VaR均值 和 TailRatio均值
        #先取出对应key的value，即一个series，在对series的元素赋值
        r_ = r['HRP'].reset_index(drop=True)
        p_ = (1 + r_).cumprod()
        stats['HRP'].loc[numIter] = p_.iloc[-1] - 1         # terminal return
        stats['HRP_VaR'].loc[numIter] = computeVaR(r_)      # VaR
        stats['HRP_TailRatio'].loc[numIter] = tail_ratio(r_)# tail_ratio derived from return series
            
        r_ = r['IVP'].reset_index(drop=True)
        p_ = (1 + r_).cumprod()
        stats['IVP'].loc[numIter] = p_.iloc[-1] - 1  
        stats['IVP_VaR'].loc[numIter] = computeVaR(r_)   
        stats['IVP_TailRatio'].loc[numIter] = tail_ratio(r_)
        
        r_ = r['EQUAL'].reset_index(drop=True)
        p_ = (1 + r_).cumprod()
        stats['EQUAL'].loc[numIter] = p_.iloc[-1] - 1  
        stats['EQUAL_VaR'].loc[numIter] = computeVaR(r_)     
        stats['EQUAL_TailRatio'].loc[numIter] = tail_ratio(r_)    
            
    #五、报告结果
    stats = pd.DataFrame.from_dict(stats , orient = 'columns')
    stats.to_csv('stats_1017_%s_%s.csv' % ( str(sigma0), str(sigma1F)))
    
    print("=================== Out-of-sample std & var =====================")
    std_of_MC ,var_of_MC = stats[['HRP','IVP','EQUAL']].std(), stats[['HRP','IVP','EQUAL']].var()
    print(std_of_MC,var_of_MC)
    print("====================== VaR Mean ======================")
    mean_of_VaR = stats[['HRP_VaR','IVP_VaR','EQUAL_VaR']].mean()
    print(mean_of_VaR)
    print("===================== TailRatio Mean =====================")
    mean_of_TR = stats[['HRP_TailRatio','IVP_TailRatio','EQUAL_TailRatio']].mean()
    print(mean_of_TR)
    
    return